In [67]:
import os
def count_no_files(dir):
    print()
    for sub in os.listdir(dir):
        print(sub + ':', end=' ')
        count = 0
        sub_dir = dir + sub
        for file in os.listdir(sub_dir):
            count += 1
        print(str(count)+' files')

count_no_files('./Data/train/fr/')
count_no_files('./Data/train/fer/')


JAMES: 139 files
JENNIE: 162 files
JISOO: 177 files
LISA: 184 files
ROSE: 196 files

ANGRY: 48 files
FEAR: 14 files
HAPPY: 75 files
NEUTROL: 40 files
SAD: 66 files
SURPRISED: 30 files


In [5]:
# train detection
import cv2
import shutil as st
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


In [10]:
def write_result_img(output_img, output_path): 
    # Đếm số lượng file hiện có trong thư mục
    count = 0
    for _, _, files in os.walk(output_path):
        count += len(files)
        
    # Tên file tiếp theo là số lượng file hiện tại + 1 
    count += 1
    if count == 7:
        print(output_path)
    output_path += "/" + str(count)+".jpg"
    check = cv2.imwrite(output_path, output_img)
    if not check:
        print("Fail to save file!", output_path)

In [78]:
def train_detection(dir):
    output = dir + "output/"
    if not os.path.exists(output):
        os.makedirs(output)
    remov_list = output + "remov_list/"
    if not os.path.exists(remov_list):
        os.makedirs(remov_list)
        
    for file in os.listdir(dir):
        file_path = dir + file
        if os.path.isdir(file_path):
            continue
        img = cv2.imread(file_path)
        bboxes = face_cascade.detectMultiScale(image=img)
        if len(bboxes)==0:
            st.copy(file_path, remov_list)
            os.remove(file_path)
            continue
        for (x,y,w,h) in bboxes:
            cv2.rectangle(img,(x,y), (x+w,y+h),(0, 255, 0), 2)
        try:
            write_result_img(img, output)
        except:
            print(file_path)
        
train_detection('./Data/train/fer/SURPRISED/')    

./Data/train/fer/SURPRISED/output/
